# 유방암 예측
_**유방 질량 이미지에서 파생된 feature 로 SageMaker의 linear-learner 를 사용하여 유방암 예측**_

---

## 차례

1. [배경](#배경)
1. [설정](#설정)
1. [데이터](#데이터)
1. [훈련](#훈련)
1. [호스팅](#호스팅)
1. [예측](#예측)
1. [확장](#확장)

---

## 배경
이 노트북은 예측을 위해 `linear models` 을 요구하는 문제해결 애플리케이션을 위해 Sagemaker 의 알고리즘을 사용하는 법을 보여줍니다. <br />
이 예에서 우리는 유방암 예측을 위해 UCI 의 유방암 진단 데이터 셋을 가져옵니다. <br />
목표는 이 데이터 세트를 사용하여 유방종 이미지가 양성 또는 악성 종양을 나타내는지 여부에 대한 예측 모델을 구축하는 것입니다. <br />

* Sagemaker 사용을 위한 기본적인 환경 구축
* Sagemaker 알고리즘이 사용할 수 있도록 데이터셋을 protobuf 포맷으로 변환하고 S3 에 업로드
* SageMaker linear learner 알고리즘으로 훈련
* 훈련된 모델을 호스팅
* 훈련된 모델을 사용하여 스코어링



---

## 설정

다음을 설정하는 것으로 시작 해보겠습니다.

* SageMaker 역할은 데이터에 대한 학습 및 호스팅 액세스 권한을 부여하는 데 사용됩니다. <br />
  아래 코드는 SageMaker 노트북 인스턴스에서 사용하는 것과 동일한 역할을 사용합니다. 그렇지 않은 경우 SageMakerFullAccess 정책이 연결된 역할의 전체 ARN을 지정합니다.
* 모델 객체를 훈련하고 저장하는 데 사용하려는 S3 버킷 지정


In [ ]:
import os
import boto3
import re
import sagemaker

role = sagemaker.get_execution_role()
region = boto3.Session().region_name

# S3 bucket for saving code and model artifacts.
# Feel free to specify a different bucket and prefix
bucket = sagemaker.Session().default_bucket()

prefix = (
    "sagemaker/DEMO-breast-cancer-prediction"  # place to upload training files within the bucket
)

이제 필요한 Python 라이브러리를 가져옵니다. 

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import io
import time
import json
import sagemaker.amazon.common as smac

---
## 데이터


> Dua, D. and Graff, C. (2019). UCI Machine Learning Repository [http://archive.ics.uci.edu/ml]. Irvine, CA: University of California, School of Information and Computer Science.

> Breast Cancer Wisconsin (Diagnostic) Data Set [https://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+(Diagnostic)].

> _Also see:_ Breast Cancer Wisconsin (Diagnostic) Data Set [https://www.kaggle.com/uciml/breast-cancer-wisconsin-data].

다음 코드는 위 데이터 소스 위치로부터 데이터를 로컬 폴더에 다운로드하여 저장한 후에 파일의 이름을 data.csv 로 변경합니다. 이 작업은 조금 시간이 소요될 수 있습니다. 

In [ ]:
s3 = boto3.client("s3")

filename = "wdbc.csv"
s3.download_file("sagemaker-sample-files", "datasets/tabular/breast_cancer/wdbc.csv", filename)
data = pd.read_csv(filename, header=None)

# 컬럼명 지정
data.columns = [
    "id",
    "diagnosis",
    "radius_mean",
    "texture_mean",
    "perimeter_mean",
    "area_mean",
    "smoothness_mean",
    "compactness_mean",
    "concavity_mean",
    "concave points_mean",
    "symmetry_mean",
    "fractal_dimension_mean",
    "radius_se",
    "texture_se",
    "perimeter_se",
    "area_se",
    "smoothness_se",
    "compactness_se",
    "concavity_se",
    "concave points_se",
    "symmetry_se",
    "fractal_dimension_se",
    "radius_worst",
    "texture_worst",
    "perimeter_worst",
    "area_worst",
    "smoothness_worst",
    "compactness_worst",
    "concavity_worst",
    "concave points_worst",
    "symmetry_worst",
    "fractal_dimension_worst",
]

# 데이터 저장
data.to_csv("data.csv", sep=",", index=False)

# 데이터 파일의 모양(shape) 프린트
print(data.shape)

# 최상위 몇개의 행 표시 
display(data.head())

# 데이터 객체를 표시
display(data.describe())

# we will also summarize the categorical field diganosis
# 필드 분류를 요약 
display(data.diagnosis.value_counts())

#### 주요 관찰 결과:
* 데이터는 569개의 관측값과 32개의 컬럼을 가지고 있습니다. 
* 첫번째 필드는 'id' 입니다.
* 두번째 필드는 'diagnosis' 이고 악성('M' = Malignant)과 양성('B' = Benign)에 대한 진단 지표입니다.
* 예측에 활용할 수 있는 30개의 다른 숫자형 feature (특성값) 들이 있습니다. 

## Features 와 Labels 생성
#### 아래 코드는 데이터를 80% 의 training 데이터, 10% 의 validation 용 데이터, 10% test 용 데이터로 나눕니다. 

In [ ]:
rand_split = np.random.rand(len(data))
train_list = rand_split < 0.8
val_list = (rand_split >= 0.8) & (rand_split < 0.9)
test_list = rand_split >= 0.9

data_train = data[train_list]
data_val = data[val_list]
data_test = data[test_list]

train_y = ((data_train.iloc[:, 1] == "M") + 0).to_numpy()
train_X = data_train.iloc[:, 2:].to_numpy()

val_y = ((data_val.iloc[:, 1] == "M") + 0).to_numpy()
val_X = data_val.iloc[:, 2:].to_numpy()

test_y = ((data_test.iloc[:, 1] == "M") + 0).to_numpy()
test_X = data_test.iloc[:, 2:].to_numpy();


이제 데이터 세트를 Amazon SageMaker 알고리즘에서 사용하는 recordIO-wrapped protobuf 형식으로 변환한 다음 이 데이터를 S3에 업로드합니다. 

In [ ]:
train_file = "linear_train.data"

f = io.BytesIO()
smac.write_numpy_to_dense_tensor(f, train_X.astype("float32"), train_y.astype("float32"))
f.seek(0)

boto3.Session().resource("s3").Bucket(bucket).Object(
    os.path.join(prefix, "train", train_file)
).upload_fileobj(f)


다음으로 유효성 검사 데이터 세트를 변환하고 업로드합니다.

In [ ]:
validation_file = "linear_validation.data"

f = io.BytesIO()
smac.write_numpy_to_dense_tensor(f, val_X.astype("float32"), val_y.astype("float32"))
f.seek(0)

boto3.Session().resource("s3").Bucket(bucket).Object(
    os.path.join(prefix, "validation", validation_file)
).upload_fileobj(f)

---
## 훈련

Amazon SageMaker의 Linear Learner는 실제로 각각 약간 다른 하이퍼파라미터를 사용하여 여러 모델을 병렬로 맞춘 다음 가장 잘 맞는 모델을 반환합니다. <br />
이 기능은 자동으로 활성화됩니다. 다음과 같은 매개변수를 사용하여 이에 영향을 줄 수 있습니다. <br />

- 'num_models' : 모델 실행의 전체 수. 최적에 가까운 솔루션을 찾기 위해 가까운 매개변수 값을 가진 모델을 선택합니다. 이번 실습에서는 최대값인 32를 사용합니다.
- 'loss' : 모델 추정치의 실수에 페널티를 부여하는 방법을 제어. 이번 실습에서는 데이터를 정리하는 데 많은 시간을 소비하지 않았으므로 absolute loss 을 사용하고 absolute loss 은 이상값 (outlier) 에 덜 민감합니다.
- 'wd' 또는 'l1' : 정규화를 제어. 정규화는 우리의 추정치가 훈련 데이터에 너무 미세하게 조정되는 것을 방지함으로써 모델 과적합을 방지할 수 있습니다. 이번 실습에서는 이 매개변수를 기본 "자동"으로 남겨둘 것입니다.


### SageMaker 의 linear-learner 를 훈련하고 호스팅하기 위해 사용되는 컨테이너 이미지 지정

In [ ]:
from sagemaker import image_uris

container = image_uris.retrieve(region=boto3.Session().region_name, framework="linear-learner")

In [ ]:
linear_job = "DEMO-linear-" + time.strftime("%Y-%m-%d-%H-%M-%S", time.gmtime())

print("Job name is:", linear_job)

linear_training_params = {
    "RoleArn": role,
    "TrainingJobName": linear_job,
    "AlgorithmSpecification": {"TrainingImage": container, "TrainingInputMode": "File"},
    "ResourceConfig": {"InstanceCount": 1, "InstanceType": "ml.c4.2xlarge", "VolumeSizeInGB": 10},
    "InputDataConfig": [
        {
            "ChannelName": "train",
            "DataSource": {
                "S3DataSource": {
                    "S3DataType": "S3Prefix",
                    "S3Uri": "s3://{}/{}/train/".format(bucket, prefix),
                    "S3DataDistributionType": "ShardedByS3Key",
                }
            },
            "CompressionType": "None",
            "RecordWrapperType": "None",
        },
        {
            "ChannelName": "validation",
            "DataSource": {
                "S3DataSource": {
                    "S3DataType": "S3Prefix",
                    "S3Uri": "s3://{}/{}/validation/".format(bucket, prefix),
                    "S3DataDistributionType": "FullyReplicated",
                }
            },
            "CompressionType": "None",
            "RecordWrapperType": "None",
        },
    ],
    "OutputDataConfig": {"S3OutputPath": "s3://{}/{}/".format(bucket, prefix)},
    "HyperParameters": {
        "feature_dim": "30",
        "mini_batch_size": "100",
        "predictor_type": "regressor",
        "epochs": "10",
        "num_models": "32",
        "loss": "absolute_loss",
    },
    "StoppingCondition": {"MaxRuntimeInSeconds": 60 * 60},
}


이제 방금 생성한 매개변수를 사용하여 SageMaker의 분산 관리 훈련작업을 시작합니다. <br />
훈련이 관리되기 때문에 작업이 끝날 때까지 기다릴 필요가 없지만 이 경우 boto3의 'training_job_completed_or_stopped' 대기자를 사용하여 작업이 시작되었는지 확인할 수 있습니다.

In [ ]:
%%time

region = boto3.Session().region_name
sm = boto3.client("sagemaker")

sm.create_training_job(**linear_training_params)

status = sm.describe_training_job(TrainingJobName=linear_job)["TrainingJobStatus"]
print(status)
sm.get_waiter("training_job_completed_or_stopped").wait(TrainingJobName=linear_job)
if status == "Failed":
    message = sm.describe_training_job(TrainingJobName=linear_job)["FailureReason"]
    print("Training failed with the following error: {}".format(message))
    raise Exception("Training job failed")

---
## 호스팅

이제 데이터에 대한 linear 알고리즘을 학습했으므로 나중에 호스팅할 수 있는 모델을 설정해 보겠습니다.

In [ ]:
linear_hosting_container = {
    "Image": container,
    "ModelDataUrl": sm.describe_training_job(TrainingJobName=linear_job)["ModelArtifacts"][
        "S3ModelArtifacts"
    ],
}

create_model_response = sm.create_model(
    ModelName=linear_job, ExecutionRoleArn=role, PrimaryContainer=linear_hosting_container
)

print(create_model_response["ModelArn"])

모델을 설정하고 나면 모델을 호스팅할 엔드포인트에 대해 설정합니다. 

1. 호스팅에 사용할 EC2 인스턴스 유형
2. 인스턴스의 초기 갯수
3. 호스팅 모델명

In [ ]:
linear_endpoint_config = "DEMO-linear-endpoint-config-" + time.strftime(
    "%Y-%m-%d-%H-%M-%S", time.gmtime()
)
print(linear_endpoint_config)
create_endpoint_config_response = sm.create_endpoint_config(
    EndpointConfigName=linear_endpoint_config,
    ProductionVariants=[
        {
            "InstanceType": "ml.m4.xlarge",
            "InitialInstanceCount": 1,
            "ModelName": linear_job,
            "VariantName": "AllTraffic",
        }
    ],
)

print("Endpoint Config Arn: " + create_endpoint_config_response["EndpointConfigArn"])

엔드포인트를 구성하는 방법을 지정했으므로 이제 엔드포인트를 생성할 수 있습니다. <br />
이것은 백그라운드에서 수행할 수 있지만 지금은 엔드포인트의 상태를 업데이트하는 루프를 실행하여 언제 사용할 준비가 되었는지 알도록 하겠습니다.

In [ ]:
%%time

linear_endpoint = "DEMO-linear-endpoint-" + time.strftime("%Y%m%d%H%M", time.gmtime())
print(linear_endpoint)
create_endpoint_response = sm.create_endpoint(
    EndpointName=linear_endpoint, EndpointConfigName=linear_endpoint_config
)
print(create_endpoint_response["EndpointArn"])

resp = sm.describe_endpoint(EndpointName=linear_endpoint)
status = resp["EndpointStatus"]
print("Status: " + status)

sm.get_waiter("endpoint_in_service").wait(EndpointName=linear_endpoint)

resp = sm.describe_endpoint(EndpointName=linear_endpoint)
status = resp["EndpointStatus"]
print("Arn: " + resp["EndpointArn"])
print("Status: " + status)

if status != "InService":
    raise Exception("Endpoint creation did not succeed")

## 예측
### 테스트 데이터를 이용한 예측

이제 호스팅된 엔드포인트가 있으므로 여기에서 통계적 예측을 생성할 수 있습니다. <br />
모델이 얼마나 정확한지 이해하기 위해 테스트 데이터 세트에서 예측해 보겠습니다.
분류 정확도를 측정하기 위한 많은 메트릭이 있습니다. 일반적인 예에는 다음이 포함됩니다. 

- Precision (정밀도)
- Recall
- F1 measure (F1 측정값)
- Area under the ROC curve - AUC
- Total Classification Accuracy (총 분류정확도)
- Mean Absolute Error (평균절대오차)

이 예에서 우리는 작업을 단순하게 유지하기 위해 Total Classification Accuracy 를 선택 지표로 사용할 것입니다. <br/>
또한 linear learner 가 이 메트릭을 사용하여 최적화되었기 때문에 MAE(Mean Absolute Error)를 평가할 것입니다. 

### array 를 csv 로 변환하기 위한 함수

In [ ]:
def np2csv(arr):
    csv = io.BytesIO()
    np.savetxt(csv, arr, delimiter=",", fmt="%g")
    return csv.getvalue().decode().rstrip()


예측값을 얻기 위해 endpoint 호출

In [ ]:
runtime = boto3.client("runtime.sagemaker")

payload = np2csv(test_X)
response = runtime.invoke_endpoint(
    EndpointName=linear_endpoint, ContentType="text/csv", Body=payload
)
result = json.loads(response["Body"].read().decode())
test_pred = np.array([r["score"] for r in result["predictions"]])


모든 인스턴스를 예측하기 위해 대다수 클래스를 사용하는 기준선 평균절대예측오류와 선형학습자 기반 평균절대예측오류를 비교합니다. <br/>
- 기준선 기반 평균절대예측오류(Baseline Mean Abolute Error) - Baseline MAE <br/>
- 선형학습자 기반 평균절대예측오류(Linear Mean Absolute Error) - Linear MAE

In [ ]:
test_mae_linear = np.mean(np.abs(test_y - test_pred))
test_mae_baseline = np.mean(
    np.abs(test_y - np.median(train_y))
)  ## training median as baseline predictor

print("Test MAE Baseline :", round(test_mae_baseline, 3))
print("Test MAE Linear:", round(test_mae_linear, 3))


예측에 대해 0.5의 분류 임계값을 사용하여 예측 정확도를 비교하고 훈련 데이터 세트로부터 다수 클래스 예측과 비교해 보겠습니다.

In [ ]:
test_pred_class = (test_pred > 0.5) + 0
test_pred_baseline = np.repeat(np.median(train_y), len(test_y))

prediction_accuracy = np.mean((test_y == test_pred_class)) * 100
baseline_accuracy = np.mean((test_y == test_pred_baseline)) * 100

print("Prediction Accuracy:", round(prediction_accuracy, 1), "%")
print("Baseline Accuracy:", round(baseline_accuracy, 1), "%")

## 자원삭제

In [ ]:
sm.delete_endpoint(EndpointName=linear_endpoint)

---
## 확장

- 우리의 linear 모델은 유방암을 잘 예측하고 92%에 가까운 전체 정확도를 가지고 있습니다. 하이퍼 파라미터, 손실 함수 등의 다른 값으로 모델을 다시 실행하고 예측이 개선되는지 확인할 수 있습니다. <br/> 
  이러한 하이퍼 파라미터에 대한 추가 조정으로 모델을 다시 실행하면 더 정확한 샘플 외 예측을 제공할 수 있습니다.
- 우리는 또한 많은 feature engineering 을 하지 않았습니다. 여러 feature 의 외적/상호작용을 고려하여 추가 feature 를 생성할 수 있습니다. 
- 추가 확장으로 XGBoost, MXNet 등과 같은 SageMaker를 통해 사용할 수 있는 많은 비선형 모델을 사용할 수 있습니다.